# Models from motions

# Purpose
Create roll decay models from the simulation results from Motions (with/without viscous damping). save these models into pkl files.

# Methodology
* Load
* Cut
* Fit
* save

# Setup

In [1]:
# %load imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import OrderedDict
#plt.style.use('paper')
from reports.paper_writing import save_fig

#import data
import copy
from mdldb.run import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import src.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

from sklearn.metrics import r2_score
from src.data import database
from mdldb import tables
import shipflowmotionshelpers.shipflowmotionshelpers as helpers
import src.visualization.visualize as visualize


Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 462 ('figure.figsize   : 5, 3   ## figure size in inches')
Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 463 ('figure.dpi       : 100        ## figure dots per inch')


In [3]:
from copy import deepcopy
import joblib
from rolldecayestimators import measure
from src.helpers import get_ikeda, calculate_ikeda, get_estimator_variation, get_data_variation , get_variation, hatify
import scipy
from sklearn.pipeline import Pipeline
from reports import mdl_results

FileNotFoundError: [Errno 2] No such file or directory: '../../data/processed/model_test_parameters.csv'

## Load data from Motions:

In [ ]:
file_paths = [
    '../data/external/kvlcc2_rolldecay_0kn',
    '../data/external/kvlcc2_rolldecay_15-5kn_const_large',
    '../data/external/kvlcc2_rolldecay_15-5kn_const_large2',
    '../data/external/kvlcc2_rolldecay_15-5kn_const_large_5deg',
    '../data/external/kvlcc2_rolldecay_15-5kn_const_large_ikeda',
    '../data/external/kvlcc2_rolldecay_15-5kn_ikeda_dev',
]
df_parameters = pd.DataFrame()
df_parameters =  helpers.load_parameters(file_path=file_paths)

mask_0 = (df_parameters['vm_s'].round(5) == 0)
df_parameters.loc[mask_0,'id'] = 21338  # MDL DB run id
df_parameters.loc[~mask_0,'id'] = 21340

In [ ]:
mask_visc = (df_parameters[['b4l','b4q']] > 0).any(axis=1)
df_parameters.loc[mask_visc]

In [ ]:
df_parameters.loc[~mask_visc]

## Load time series

In [ ]:
def find_t_max(X, phi_max):

    Xs = X.copy()
    Xs.index = pd.TimedeltaIndex(X.index, unit='s')
    X_ = Xs.abs().resample('5S').max()
    X_.index = X_.index.total_seconds()
    mask = X_['phi'] < phi_max
    
    X_small = X_.loc[mask]
    if len(X_small) > 0:
        t_max = X_small.iloc[0].name
    else:
        t_max = X.index[-1]
    
    return t_max

In [ ]:
time_series = helpers.load_time_series(df_parameters=df_parameters)
time_series_raw = deepcopy(time_series)

phi_max = np.deg2rad(3.5)

for key,df in time_series.items():
    
    if df.mean().abs()['V1'] > 0.01:
        
        phi1d_limit = 10**-2
        index0 = (df['phi1d'].abs() > phi1d_limit).argmax()
        X = df.iloc[index0:].copy()
        t_max = find_t_max(X=X, phi_max=phi_max)
        mask = X.index <= t_max
        X = X.loc[mask]
        
        time_series[key] = X

## Save raw datasets 

In [ ]:
interesting = ['phi','phi1d','phi2d','V1']

save_runs = [
'kvlcc2_rolldecay_0kn',
'kvlcc2_rolldecay_15-5kn_const_large2',
'kvlcc2_rolldecay_15-5kn_ikeda_dev',
]

for key in save_runs:
    
    df_save = time_series[key][interesting].copy()
    save_file_name = '%s.csv' % key
    save_path = os.path.join('../data/processed/', save_file_name)
    df_save.to_csv(save_path)
    
df_parameters_save = df_parameters.loc[save_runs].copy()
save_path = os.path.join('../data/processed/', 'fnpf_parameters.csv')
df_parameters_save.to_csv(save_path)


df_rolldecays = mdl_results.df_rolldecays.copy()
save_path = os.path.join('../data/processed/', 'model_test_parameters.csv')
df_rolldecays.to_csv(save_path)

## Cutting

In [ ]:
#X = time_series['kvlcc2_rolldecay_15-5kn_const_large_ikeda']
#time_series['kvlcc2_rolldecay_15-5kn_const_large_ikeda'] = X.loc[0:110]

In [ ]:
for key, df in time_series.items():
    
    fig,ax = plt.subplots()
    fig.set_size_inches(20,4)
    time_series_raw[key].plot(y='phi', ax=ax)
    df.plot(y='phi', ax=ax)
    ax.set_title(key)

## Fitting

In [ ]:
models = OrderedDict()

for key,X in time_series.items():
    
    pre_model= EstimatorQuadraticB(fit_method='derivation')
    pre_model.fit(X)
    
    model_motions = EstimatorQuadraticB(p0=pre_model.parameters)
    
    try:
        model_motions.fit(X=X)
    except scipy.linalg.LinAlgError:
        model_motions.fit(X=X)  # Retry
    
    if pre_model.score() > model_motions.score():
        model_motions = pre_model
        
    models[key] = model_motions

## Genereate results

In [ ]:
for key, model in models.items():
    
    parameters = df_parameters.loc[key]
    
    row = mdl_results.df_rolldecays.loc[parameters.id]
    
    scale_factor = row.scale_factor
    meta_data = {
        'Volume' : row.Volume/(scale_factor**3),
        'rho' : row.rho,
        'g' : row.g,
        'GM' : row.gm/scale_factor,
        }

    model.result_for_database(meta_data=meta_data)  # Results are stored in object
    
    if not 'B_3' in model.results:
        model.results['B_3'] = 0
    

## Saving models

In [ ]:
for key,model in models.items():
    
    # Saving the model as a pipline:
    pipeline = Pipeline(steps=[('estimator',model)])
    
    joblib.dump(pipeline, '../models/%s.pkl' % key)
    

In [ ]:
for key,model in models.items():
    
    model.plot_damping()
    